# Importação da Bibliotecas

In [80]:
import pandas as pd
import numpy as np
import psycopg2
from psycopg2 import OperationalError
from io import StringIO
import seaborn as sns
from scipy.stats.stats import pearsonr
import csv

# Configura para o número de casas decimais que serão informadas após a vírgula.
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Funções

In [81]:
# Cria a conexão com a bases de dados, necessário para execução das consultas.
def create_connection(db_name, db_user, db_password, db_host, db_port):
    connection = None
    try:
        connection = psycopg2.connect(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port,
        )
        print("Connection to PostgreSQL DB successful")
    except OperationalError as e:
        print(f"The error '{e}' occurred")
    return connection

# Execução das consultas no Banco de Dados.            
def execute_query(connection, query):
    connection.autocommit = True
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Query executed successfully")
    except OperationalError as e:
        print(f"The error '{e}' occurred")
        
# Carga dos dados na base de dados.    
def copy_from_stringio(conn, df, table):
    """
    Here we are going save the dataframe in memory 
    and use copy_from() to copy it to the table
    """
    # save dataframe to an in memory buffer
    buffer = StringIO()
    df.to_csv(buffer, index=False, header=False,sep=';')
    buffer.seek(0)
    
    cursor = conn.cursor()
    try:
        cursor.copy_from(buffer, table, sep=";")
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("copy_from_stringio() done")
    cursor.close()

# Extração dos Dados

### Leitura dos dados

In [82]:
data = pd.read_csv(r'C:\Users\elwes\OneDrive\Documentos\Projetos\IBGE - PENSE\2015\Amostra 1\arquivos csv\PENSE_AMOSTRA1_ALUNOESCOLA.CSV',sep=';')

path_dicionario_alunos = r'C:\Users\elwes\OneDrive\Documentos\Projetos\IBGE - PENSE\2015\Amostra 1\Dicionario_PENSE_Microdados_Amostra1.xls'
cabecalho_dicionario_alunos = pd.read_excel(path_dicionario_alunos,sheet_name='AMOSTRA1_ALUNO')

path_dicionario_escolas = r'C:\Users\elwes\OneDrive\Documentos\Projetos\IBGE - PENSE\2015\Amostra 1\Dicionario_PENSE_Microdados_Amostra1.xls'
cabecalho_dicionario_escolas = pd.read_excel(path_dicionario_escolas,sheet_name='AMOSTRA1_ESCOLA')

# Transformação dos Dados
A príncipio serão realizadas as transformações necessárias para que os dados da tabela de amostras tenham correspondentes descritivos em tabelas que serão posteriormente no sistema de visualização.<br/>
Portanto os dados serão divididos em dois grupos:<br/>

dados_dicionario_XXXX = Corresponde aos valores únicos possíveis para cada variável.<br/>
cabecalho_dicionario_XXXX = Corresponde ao descritivo para cada um dos valores únicos das variáveis. 

In [83]:
dados_dicionario_alunos = cabecalho_dicionario_alunos.copy()
dados_dicionario_escolas = cabecalho_dicionario_escolas.copy()

## Dicionários

### Dicionário dos Cabeçalhos

In [84]:
# Coluna criada para posterior classificação, colunas onde o valor True for atribuído serão entendidas como cabeçalho.
cabecalho_dicionario_alunos['Filtro'] = False
cabecalho_dicionario_alunos['Respondente'] = 'Aluno'
# Caso o valor do primeiro caractere não seja um número de 0 à 10 ou '-', este será um cabeçalho.
for i,var in enumerate(cabecalho_dicionario_alunos['VARIÁVEL']):
    var = str(var)
    var_s = [x for x in var]
    if var_s[0] not in ['1','2','3','4','5','6','7','8','9','0','-']:
        cabecalho_dicionario_alunos.loc[i,['Filtro']] = True      
# Reset dos índices, filtro das linhas onde 'Filtro' == True e seleção das colunas.
cabecalho_dicionario_alunos.rename({'QUESTIONÁRIO DO ALUNO':'QUESTIONÁRIO'}, axis=1, inplace=True)       
cabecalho_dicionario_alunos = cabecalho_dicionario_alunos.loc[cabecalho_dicionario_alunos['Filtro'] == True,['VARIÁVEL','QUESTIONÁRIO','Respondente']] 

In [85]:
# Coluna criada para posterior classificação, colunas onde o valor True for atribuído serão entendidas como cabeçalho.
cabecalho_dicionario_escolas['Filtro'] = False
cabecalho_dicionario_escolas['Respondente'] = 'Escola'
# Caso o valor do primeiro caractere não seja um número de 0 à 10 ou '-', este será um cabeçalho.
for i,var in enumerate(cabecalho_dicionario_escolas['VARIÁVEL']):
    var = str(var)
    var_s = [x for x in var]
    if var_s[0] not in ['1','2','3','4','5','6','7','8','9','0','-']:
        cabecalho_dicionario_escolas.loc[i,['Filtro']] = True 
# Reset dos índices, filtro das linhas onde 'Filtro' == True e seleção das colunas.
cabecalho_dicionario_escolas.rename({'QUESTIONÁRIO DA ESCOLA':'QUESTIONÁRIO'}, axis=1, inplace=True)       
cabecalho_dicionario_escolas = cabecalho_dicionario_escolas.loc[cabecalho_dicionario_escolas['Filtro'] == True,['VARIÁVEL','QUESTIONÁRIO','Respondente']]

#### Junção dos Cabeçalhos

In [86]:
# Loop para iterar em todas as colunas criando uma lista com o nome da coluna e descricao.
lista_colunas_dic_dados = []
cabecalho_dicionario_completo = pd.concat([cabecalho_dicionario_alunos, cabecalho_dicionario_escolas]).reset_index()
cabecalho_dicionario_completo.drop('index', axis=1, inplace=True)
for i in cabecalho_dicionario_completo['VARIÁVEL']:
    lista_colunas_dic_dados.append(i)
    lista_colunas_dic_dados.append("descricao")

### Dicionário dos Dados

In [87]:
# Ajuste dos nomes das colunas
dados_dicionario_alunos.rename({'VARIÁVEL':'Variável_1','QUESTIONÁRIO DO ALUNO':'Questionário'}, axis=1, inplace=True)
dados_dicionario_escolas.rename({'VARIÁVEL':'Variável_1','QUESTIONÁRIO DA ESCOLA':'Questionário'}, axis=1, inplace=True)

#### Junção dos Dados do Dicionário da Escola e Aluno

In [88]:
# Lista para criação do Dicionário de Dados
dados_dicionario_raw = pd.concat([dados_dicionario_alunos,dados_dicionario_escolas]).reset_index()
dados_dicionario_raw.drop('index', axis=1, inplace=True)

In [89]:
# Dicionário vazio com 5000 linhas
dicionario_dados_completo = pd.DataFrame(index=range(0,5000),columns = lista_colunas_dic_dados)
lista_null = dicionario_dados_completo.iloc[:,0].copy()

In [90]:
coluna_dado = -2
coluna_descricao = -1
linha = -1

# O loop abaixo ira iterar entre as tabelas dicionario_dados_completo e dados_dicionario_raw, onde teremos o link entre 
# o nome da variável, seu valor na tabela da pesquisa e descrição.
for i,var in enumerate(dados_dicionario_raw['Variável_1']):
    var = str(var)
    var_s = [x for x in var]
    if var_s[0] not in ['1','2','3','4','5','6','7','8','9','0','-']:
        coluna_dado = coluna_dado + 2
        coluna_descricao = coluna_descricao + 2        
        linha = 0
    dicionario_dados_completo.iloc[linha,coluna_dado]      = dados_dicionario_raw['Variável_1'][i]
    dicionario_dados_completo.iloc[linha,coluna_descricao] = dados_dicionario_raw['Questionário'][i]
    linha = linha + 1
dicionario_dados_completo = dicionario_dados_completo[1:]
dicionario_dados_completo = dicionario_dados_completo.copy().reset_index()
dicionario_dados_completo.drop('index', axis=1,inplace=True)
dicionario_dados_completo

,ANOPESQ,descricao,PAIS,descricao,REGEOGR,descricao,UFCENSO,descricao,MUNICIPIO_CAP,descricao,...,ESTRATO_EXP,descricao,PESO,descricao,aluno,descricao,escola,descricao,turma,descricao
0,NaN,NaN,76,Brasil,1,Norte,11,Rondônia,0,Não é capital,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,2,Nordeste,12,Acre,1100205,Porto Velho,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,3,Sudeste,13,Amazonas,1200401,Rio Branco,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,4,Sul,14,Roraima,1302603,Manaus,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,5,Centro-Oeste,15,Para,1400100,Boa Vista,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
# Algumas colunas não possuem intervalos de valores com uma descrição atríbuida, para estes casos, será iterado na "dicionario_dados_completo" buscando por esta condição
# e onde  for atendida a mesma seá preenchida com os valores únicos encontrados na coluna correpondente da tabela "dados".
coluna_dado = 0
coluna_descricao = 1
linha = -1
for i,var in enumerate(lista_colunas_dic_dados):
    lista_null_iter = lista_null.copy()
    posicao_zero = pd.DataFrame(dicionario_dados_completo[var]).iloc[0,0]
#   A coluna 'descricao' será ignorada pois não trata da variável propriamente dita.
#   As colunas de 'aluno' e 'turma' possuem muitos registros o que torna esta verificação muito demorada e como neste caso serão objeto de estudo, estas não serão verificadas.
    excecoes = ['descricao', 'aluno', 'turma' ]
    if var in excecoes:
        pass
    elif posicao_zero is np.nan:
        valores = list(data[var].unique())
        for j,var_uni in enumerate(valores):
            dicionario_dados_completo.loc[j,var] = var_uni


### Ponto de partida para Análise </br>
Àfim de definir um ponto de partida para início de pontos relevantes e suas possíveis correlações, foi elaborada a correlação entre todas as variáveis do estudo.

In [92]:
correlacao = data.corr()
correlacao.reset_index(inplace=True)
correlacao.rename({'index':'VARIÁVEL'}, axis=1, inplace=True)
correlacao.to_csv(r'C:\Users\elwes\OneDrive\Documentos\Projetos\IBGE - PENSE\2015\Amostra 1\df_correlacao_questionario.CSV')

df_correlacao_questionario = correlacao.melt(    id_vars = ['VARIÁVEL'],
                                    var_name = ['VARIÁVEL_2'],
                                    value_name = "correlacao")
df_correlacao_questionario = df_correlacao_questionario.dropna()
df_correlacao_questionario = df_correlacao_questionario.loc[df_correlacao_questionario['VARIÁVEL'] != df_correlacao_questionario['VARIÁVEL_2'],]
df_correlacao_questionario = df_correlacao_questionario.sort_values(['correlacao'])
df_correlacao_questionario = pd.merge(df_correlacao_questionario,cabecalho_dicionario_completo[['VARIÁVEL','QUESTIONÁRIO']], on=['VARIÁVEL'],how='left')
cabecalho_dicionario_completo.rename({'VARIÁVEL':'VARIÁVEL_2'}, axis=1, inplace=True)
df_correlacao_questionario = pd.merge(df_correlacao_questionario,cabecalho_dicionario_completo[['VARIÁVEL_2','QUESTIONÁRIO']], on=['VARIÁVEL_2'],how='left')
cabecalho_dicionario_completo.rename({'VARIÁVEL_2':'VARIÁVEL'}, axis=1, inplace=True)
df_correlacao_questionario

,VARIÁVEL,VARIÁVEL_2,correlacao,QUESTIONÁRIO_x,QUESTIONÁRIO_y
0,VE01P34,VE01P35,-0.933,A escola fica aberta nos finais de semana para...,"As ações desenvolvidas na escola, no final de ..."
1,VE01P35,VE01P34,-0.933,"As ações desenvolvidas na escola, no final de ...",A escola fica aberta nos finais de semana para...
2,VE01P13,VE01P14A11,-0.903,Existe algum ponto alternativo de venda de pro...,O ponto alternativo vende frutas frescas ou sa...
3,VE01P14A11,VE01P13,-0.903,O ponto alternativo vende frutas frescas ou sa...,Existe algum ponto alternativo de venda de pro...
4,VE01P37,VE01P21,-0.898,A escola tem vestiários separados para alunos ...,A escola tem vestiário EM CONDIÇÕES DE USO par...
...,...,...,...,...,...
88483,UFCENSO,ESTRATOGEOREG,1.000,Unidade da Federação,Indicador de estrato georeg
88484,UFCENSO,ESTRATOGEOREG,1.000,Unidade da Federação,Indicador de estrato georeg
88485,UFCENSO,ESTRATOGEOREG,1.000,Unidade da Federação,Indicador de estrato georeg
88486,VB03011A,TEMPOEST,1.000,"NOS ÚLTIMOS 7 DIAS, em quantos dias você fez a...",A atividade física globalmente estimada refere...


## Criação das Tabelas Dimensão em SQL
Nessa etapa todas as tabelas dimensão foram exportadas para arquivos individuais e o comando SQL para criação das tabelas foi elaborado

In [93]:
# Variável 'sql_create_d_tables' onde serão armazenados os comandos para criação de todas as tabelas dimensão extraídos do dicionário.
sql_create_d_tables = ''
# Lista com o nome de todas as tabelas à serem carregadas no banco de dados.
list_of_d_tables = []
# Lista com os Dataframes à serem carregados no banco de dados
dfs_d_tables = []
# O Dataframe da posição dfs_d_tables[0] corresponde ao nome list_of_d_tables[0], essa relação será utilizada para criação do loop
# de criação dos comandos SQL.

# Caminho para salvar as tabelas dimensão em .CSV, não serão utilizadas posteriormente neste pois serão utilizadas as tabelas já
# carregadas durante a execução
tabela_dic_path = 'C:\\Users\\elwes\\OneDrive\\Documentos\\Projetos\\IBGE - PENSE\\2015\\Amostra 1\\tabelas_dicionarios\\'

# Posição inicial para iniciar criação das tabelas dimensão para utilizar no While à seguir.
i = 0 # o Índice 'i' ficará responsável por selecionar as variáveis 
# Loop para iterar em todas as colunas do colunas do Dicionario de Dados
while i < len(dicionario_dados_completo.columns):
    j = i + 2 # o Índice 'j' será responsável por selecionar a coluna de descrição correspondente a variável
    # Lembrete: Quando o range i:j é selecionado temos na posição inicial "0:2", o padrão de seleção é 'up to but not included', ou seja,
    # a coluna 2 não é selecionada, assim temos a seleção das colunas 0:1.
    temp = dicionario_dados_completo.iloc[:,i:j] # Tabela contendo apenas os dados referentes a variável 'dicionario_dados_completo.columns[i]'
    temp.dropna(axis=0, how='all', inplace=True) # Remoção das linhas com valores nulos
    temp.reset_index() # Reset do index
    temp.to_csv(tabela_dic_path + temp.columns[0] + '.csv' ) # Exportação para um arquivo .CSV
    # Temos variáveis comuns no dicionario Escola e Aluno portanto, no momento da elaboração da lista de à serem carregas no banco de dados
    # precisamos incluir os dados apenas uma vez, dessa forma a condicação if seguinte verifica se a variável em questão já foi incluída na lista
    # em caso afirmativo passamos ao próximo loop.
    if temp.columns[0] in list_of_d_tables:
        i = i + 2 # Incremento na indice selecionador de colunas.
        continue
    # Lista com o nome dos Dataframes
    list_of_d_tables.append(temp.columns[0])
    # Lista com os Dataframes, cada posição da lista corresponde à um Dataframe.
    dfs_d_tables.append(temp)
    
    # Comando SQL para criação das tabelas dimensão. 
    sql_create_d_tables= sql_create_d_tables + 'CREATE TABLE IF NOT EXISTS ' + temp.columns[0] + ' ( \n'
    var_type = 'TEXT' 
    for j, column in enumerate(temp.columns):
        if j < len(temp.columns)-1:
            sql_create_d_tables = sql_create_d_tables + column + '\t' + var_type + ',\n'
        else :  
            sql_create_d_tables = sql_create_d_tables + column + '\t' + var_type + '\n'
    sql_create_d_tables = sql_create_d_tables + ');\n\n'    
    i = i + 2 # Incremento na indice selecionador de colunas.
# Exportação do comando SQL para um arquivo .txt   
with open('sql_create_d_tables.txt', mode='w') as txtfile:
    txtfile.write(sql_create_d_tables)

## Criação Base de Dados AWS

In [ ]:
connection = create_connection('postdb',
                                  'elweshonorato',
                                  'Andromeda=2121',
                                  'main.czqwwerei65b.us-east-2.rds.amazonaws.com',
                                  '5432')

create_db_pense_ibge_query = "CREATE DATABASE pense_ibge"
execute_query(connection, create_db_pense_ibge_query)

## Criação das Tabelas Dimensão

In [ ]:
connection = create_connection('pense_ibge',
                               'elweshonorato',
                               'Andromeda=2121',
                               'main.czqwwerei65b.us-east-2.rds.amazonaws.com',
                               '5432')

execute_query(connection, sql_create_d_tables)

## Carga dos Dados nas Tabelas Dimensão

In [ ]:
connection = create_connection('pense_ibge',
                               'elweshonorato',
                               'Andromeda=2121',
                               'main.czqwwerei65b.us-east-2.rds.amazonaws.com',
                               '5432')

for i,df in enumerate(list_of_d_tables):
    copy_from_stringio(connection, dfs_d_tables[i],     df)

## Criação da Tabela Fato

In [ ]:
sql_create_f_tables = ''
sql_create_f_tables= sql_create_f_tables + 'CREATE TABLE IF NOT EXISTS f_dados_pesquisa' + ' ( \n'
var_type = 'TEXT'
for j, column in enumerate(data.columns):
    if j < len(data.columns)-1:
        sql_create_f_tables = sql_create_f_tables + column + '\t' + var_type + ',\n'
    else :  
        sql_create_f_tables = sql_create_f_tables + column + '\t' + var_type + '\n'
        sql_create_f_tables = sql_create_f_tables + ');\n\n'
    
with open('sql_create_f_tables.txt', mode='w') as txtfile:
    txtfile.write(sql_create_f_tables)

## Carga dos Dados na Tabela Fato

In [ ]:
connection = create_connection('pense_ibge',
                               'elweshonorato',
                               'Andromeda=2121',
                               'main.czqwwerei65b.us-east-2.rds.amazonaws.com',
                               '5432')
execute_query(connection, sql_create_f_tables)
copy_from_stringio(connection, data, 'f_dados_pesquisa')